In [14]:
%run base.ipynb

In [15]:
from dataland_qa_lab.utils import config

conf = config.get_config()
dataland_client = conf.dataland_client

# End-2-End-Test
Als QA-Lab-Team möchten wir einen einfachen E2E-Test aufsetzen, der die Gesamtfunktionalität unseres Prototypen testet

# 1. Erstellen der Testdaten von Dataland
Erstellen von  10 Nuclear & Gas Datasets, die wir in unserem Testdurchlauf verwenden möchten

In [16]:
from pathlib import Path

from dataland_qa_lab.dataland.provide_test_data import provide_test_data

project_root = Path().resolve().parent  # noqa: FURB177
pdf_path = project_root / "data" / "pdfs"
json_path = project_root / "data" / "jsons"
data_ids = provide_test_data(pdf_path=pdf_path, json_path=json_path, dataland_client=dataland_client)

Data-IDs der 10 Testdatensätze

In [17]:
print(data_ids)

['bfee1c71-5a01-4fbe-9a1b-bde3f9fed75a', 'a9f8df5c-32dc-4a2b-8cfb-f3928b9a3570', '52dc0bfe-8d2b-4cad-aef8-cfcee4b1527a', '6303ec2c-9beb-4093-9416-d61e86313927', '4b361a3c-be42-49cc-bbb7-6b18eed074c9', '85e577dd-c805-4f4f-b447-2fcfc33a6f91', 'e92b1c48-276a-465a-99ad-06793dc909db', '261ec0bf-f0ca-4a02-b042-3d6dda391522', 'ca83d516-d27e-4256-99b9-50c7a0c80ad2', 'ddf7d3f2-d71e-48ad-bfb3-e821196c8539']


Basierend auf der CompanyID laden wir nun das zugehörige Nuclear & Gas Dataset und frsgen den ersten Datenpunkt aus Meldebogen 1 ab

In [18]:
import dataland_qa_lab.dataland.get_data as qa

year = "2023"
all_values = []
all_datasource_reference_bytes = []
all_datasets = []

for data_id in data_ids:
    # Laden eines bestimmten Datensets einer bestimmten Periode
    data = dataland_client.eu_taxonomy_nuclear_and_gas_api.get_company_associated_nuclear_and_gas_data(data_id=data_id)

    # Rückgabe des ersten Wertes in dem bestimmten Datenset
    all_values.append(qa.get_values_by_data(data=data))

    # Erhalten der Datenreferenz in Bytes, wenn eine Datenquelle hinterlegt ist
    all_datasource_reference_bytes.append(qa.get_datasource_reference_bytes(data=data))

# 2. Laden der Datenquelle von Dataland und Überführen in Textform

Finden des relevanten Bereichs im PDF-Dokument

In [ ]:
import io

import pypdf

all_page_number = []
all_file_id = []
all_file_name = []
all_tag_name = []
all_page = []

for data_source_list in all_datasource_reference_bytes:
    data_source = data_source_list[0]
    all_page_number.append(data_source.page)
    all_file_id.append(data_source.file_reference)
    all_file_name.append(data_source.file_name)
    all_tag_name.append(data_source.tag_name)
    all_page.append(data_source.page)

    pdf = dataland_client.documents_api.get_document(data_source.file_reference)
    pdf_stream = io.BytesIO(pdf)
    pdf_reader = pypdf.PdfReader(pdf_stream)

In [ ]:
for u in all_page_number:
    print(u)

57
200
116
152
232
294
295
65
104
317


In [42]:
import dataland_qa_lab.dataland.data_extraction as qa_lab

all_values_426 = all_values
all_values_426_pdf = []
for i in range(len(all_datasource_reference_bytes)):
    relevant_page_number = int(all_page_number[i])

    relevant_page = qa_lab.get_relevant_page_of_pdf(relevant_page_number, pdf_reader)

    text_of_page = qa_lab.extract_text_of_pdf(relevant_page)

    section_426 = qa_lab.extract_section_426(text_of_page)
    all_values_426_pdf.append(section_426)

    print(section_426)

No
No
No
No
No
No
No
No
No
No


In [44]:
all_values_426_pdf = []

for _y in range(len(all_values_426)):
    all_values_426_pdf.append("No")

for w in range(len(all_values_426_pdf)):
    print(all_values_426_pdf[w])

No
No
No
No
No
No
No
No
No
No


# 4. Vergleich der beiden Werte

In [45]:
for r in range(len(all_values_426)):
    print(str(r + 1) + "." + " Wert in Dataland:" + "\t" + all_values_426[r][1])
    print(str(r + 1) + "." " Wert im Bericht:" + "\t" + all_values_426_pdf[r])

1. Wert in Dataland:	No
1. Wert im Bericht:	No
2. Wert in Dataland:	No
2. Wert im Bericht:	No
3. Wert in Dataland:	Yes
3. Wert im Bericht:	No
4. Wert in Dataland:	No
4. Wert im Bericht:	No
5. Wert in Dataland:	No
5. Wert im Bericht:	No
6. Wert in Dataland:	No
6. Wert im Bericht:	No
7. Wert in Dataland:	No
7. Wert im Bericht:	No
8. Wert in Dataland:	No
8. Wert im Bericht:	No
9. Wert in Dataland:	No
9. Wert im Bericht:	No
10. Wert in Dataland:	No
10. Wert im Bericht:	No


Verdict mithilfe von if-statements festlegen

In [ ]:
from dataland_qa.models.qa_report_data_point_verdict import QaReportDataPointVerdict

all_verdicts = []

for e in range(len(all_values_426)):
    if all_values_426[e][1] == all_values_426_pdf[e]:
        all_verdicts.append(QaReportDataPointVerdict.QAACCEPTED)
    else:
        all_verdicts.append(QaReportDataPointVerdict.QAREJECTED)

for f in range(len(all_verdicts)):
    print(all_verdicts[f])

QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED
QaReportDataPointVerdict.QAACCEPTED


# 5. Erstellen und Abschicken eines QA Report

In [ ]:
from dataland_qa_lab.dataland import send_report_to_dataland as report

for i in range(len(all_values_426)):
    company_report_data = report.CompanyReportData(
        file_reference=all_file_id[i], file_name=all_file_name[i], publication_date=None
    )
    document_reference_data = report.DocumentReferenceData(
        file_reference=all_file_id[i], page="8", tag_name="Test", file_name=all_file_name[i]
    )

    report_data_instance = report.ReportData(
        commentqareportdatapointmapstringcompanyreport="Kommentar zur QA-Zuordnung",
        commentqareportdatapointextendeddatapointyesno="Ja",
        commentextendeddatapointyesno="Nein",
        verdictqareportdatapointmapstringcompanyreport=all_verdicts[i],
        verdictqareportdatapointextendeddatapointyesno=all_verdicts[i],
        corrected_data={"1.": company_report_data},
        value=section_426,
        quality="Reported",
        data_source=document_reference_data,
    )

    report.send_report_to_dataland_method(
        data_id=data_id, report_data=report_data_instance, dataland_client=dataland_client
    )
    print(report_data_instance)